In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # CIFAR-100均值和標準差
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

print("訓練圖像:")
imshow(torchvision.utils.make_grid(images[:8]))
print('標籤:', labels[:8].numpy())

dataiter = iter(test_loader)
images, labels = next(dataiter)

print("測試圖像:")
imshow(torchvision.utils.make_grid(images[:8]))
print('標籤:', labels[:8].numpy())


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Linear(512, 100)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 50
training_losses = []
testing_losses = []
training_accuracies = []
testing_accuracies = []

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_training_loss = running_loss / len(train_loader)
    training_losses.append(avg_training_loss)
    training_accuracy = 100 * correct_train / total_train
    training_accuracies.append(training_accuracy)
    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {avg_training_loss:.4f}, Training Accuracy: {training_accuracy:.2f}%')

    # Testing phase
    model.eval()
    testing_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            testing_loss += loss.item()

            # Calculate testing accuracy
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    avg_testing_loss = testing_loss / len(test_loader)
    testing_losses.append(avg_testing_loss)
    testing_accuracy = 100 * correct_test / total_test
    testing_accuracies.append(testing_accuracy)
    print(f'Epoch [{epoch+1}/{epochs}], Testing Loss: {avg_testing_loss:.4f}, Testing Accuracy: {testing_accuracy:.2f}%')

In [ ]:
# Final Accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on CIFAR-100: {100 * correct / total:.2f}%')

# Plot losses and accuracies
import matplotlib.pyplot as plt

# Plot Training and Testing Losses
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), training_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), testing_losses, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Testing Loss')

# Plot Training and Testing Accuracies
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), training_accuracies, label='Training Accuracy')
plt.plot(range(1, epochs + 1), testing_accuracies, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Training and Testing Accuracy')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])

model.eval()
num_images = 30
images_shown = 0

with torch.no_grad():
    for images, labels in test_loader:
        if images_shown >= num_images:
            break

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        for i in range(images.size(0)):
            if images_shown < num_images:
                image = images[i].cpu().numpy().transpose((1, 2, 0))
                image = image * std + mean
                image = np.clip(image, 0, 1)

                plt.subplot(5, 6, images_shown + 1)
                plt.imshow(image)
                plt.axis('off')

                true_class = labels[i].item()
                predicted_class = predicted[i].item()
                plt.title(f'True: {true_class}, Pred: {predicted_class}', fontsize=8)

                images_shown += 1

    plt.tight_layout()
    plt.show()


In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')